In [1]:
# this is going to act as our document for the RAG model to learn from. 
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore new cuisine by dining at an athnic restuarant.",
    "Take a yoga class."
]

In [3]:
# this is the user query
user_query = "I am an Indian and I live in India "

In [4]:
# here we perform a similarity search between the below document and query
document="India is my country for the Indians and for everyone test"

In [5]:
from collections import Counter
import math

In [6]:
query_tokens=user_query.lower().split(" ")

In [7]:
document_tokens=document.lower().split(" ")

In [8]:
# query_tokens
# document_tokens
# above are the tokens created from the user query and example document

In [9]:
# here we calculate the frequency of every token in the query and doc to use them as embeddings for the tokens
query_counter = Counter(query_tokens)
document_counter = Counter(document_tokens)

In [10]:
# this is a sentence vector
lst = []  #this list holds the frequency values of all tokens of user query
for token in query_counter.keys():
    print(query_counter[token], token)
    lst.append(query_counter[token])
lst

2 i
1 am
1 an
1 indian
1 and
1 live
1 in
1 india
1 


[2, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
doc = []
for token in document_counter.keys():
    doc.append(document_counter[token])

In [12]:
# this will give us the commons words in both the sentences
mylist=[]
for tokens in query_counter.keys() & document_counter.keys():
    print(tokens)
    mylist.append(query_counter[tokens]*document_counter[tokens])

india
and


In [13]:
# this is the dot product of those two common words
dot_prod=sum(mylist)

In [25]:
query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))

In [27]:
query_magnitude

3.4641016151377544

In [29]:
document_magnitude = math.sqrt(sum(document_counter[token] ** 2 for token in document_counter))
document_magnitude

3.605551275463989

In [31]:
similarity = (dot_prod)/(query_magnitude*document_magnitude)
similarity

0.16012815380508716

In [33]:
#user_query="is yoga good for health"
#document="yoga is very good for living healthy lifestyle"

In [35]:
def cosine_similarity(query, document):
    query_tokens=query.lower().split(" ")
    document_tokens=document.lower().split(" ")

    query_counter = Counter(query_tokens)
    document_counter = Counter(document_tokens)

    mylist=[]
    for tokens in query_counter.keys() & document_counter.keys():
          print(tokens)
          mylist.append(query_counter[tokens]*document_counter[tokens])
    dot_prod=sum(mylist)
    query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))
    document_magnitude = math.sqrt(sum(document_counter[token] ** 2 for token in document_counter))
    similarity = (dot_prod)/(query_magnitude*document_magnitude)

    return similarity

In [37]:
#cosine_similarity(user_query, document)

In [39]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = cosine_similarity(query, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [41]:
corpus_of_documents

['Take a leisurely walk in the park and enjoy the fresh air.',
 'Visit a local museum and discover something new.',
 'Attend a live music concert and feel the rythm.',
 'Go for a hike and admire the natural scenery.',
 'Have a picnic with friends and share some laughs.',
 'Explore new cuisine by dining at an athnic restuarant.',
 'Take a yoga class.']

In [43]:
new_query="i like fresh air."

In [45]:
return_response(new_query, corpus_of_documents)

air.
fresh


'Take a leisurely walk in the park and enjoy the fresh air.'

In [47]:
another_query="I like museum"
return_response(another_query, corpus_of_documents)

museum


'Visit a local museum and discover something new.'

In [49]:
#how to configure LLM in the local system
#using LLAMA2
#we won't be using hugging face 

In [51]:
relevant_document = return_response(new_query, corpus_of_documents)
user_input=new_query

air.
fresh


In [56]:
#augment this response above using the llama2 model (connect the llama2 model and integrate it with our own model)

import requests
import json
full_response = []
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

url = 'http://localhost:11434/api/generate'

data={
    "model":"llama2",
    "prompt":prompt.format(user_input=user_input, relevant_document=relevant_document)
}

headers={'Content-Type':'application/json'}

response=requests.post(url, data=json.dumps(data), headers=headers, stream=True)
try:
    for line in response.iter_lines():
        if line:
            decode_line = json.loads(line.decode('utf-8'))
            full_response.append(decode_line['response'])
finally:
    response.close()
print(''.join(full_response))


 Great! Based on your interest in fresh air, I recommend taking a leisurely walk in the park to enjoy the natural surroundings.
